#### Random Graph Generation

In [1]:
import numpy as np
import random
import pickle
import os

K=[10]
nV=[200]                      #nv is the number of vertices in the graph.  
nC=[20]                      #cv is the number of clients for each facility
num_graphs=1
penalty=10000

modes=['skewed[1-10]']
da=['training','testing']
# mode='/skewed[1-6]'
# mode='/skewed[1,10]'
for nv in nV:
    for nc in nC:
        for mode in modes:
            os.system('mkdir'+' data/')
            os.system('mkdir'+' data/'+mode)
            os.system('mkdir'+' data/'+mode+'/V='+str(nv)+'-C='+str(nc))
            for ki,k in enumerate(K):
                os.system('mkdir'+' data/'+mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k))
                os.system('mkdir'+' data/'+mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/testing')
                os.system('mkdir'+' data/'+mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/training')
                print(nv,nc,mode,k)
                for d in da:
                    path='data/'+mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/'+d+'/'
                    for graph in range(num_graphs):
                        adj=np.zeros([nv*nc,3])
                        new_pairs=[]
                        for i in range(1,nv+1):
                            y=np.sort(random.sample(range(1,nv+1), nc))
                            for j in (y):
                                new_pairs.append((i,j))
                        new_pairs=np.array(new_pairs)


                        adj[:,0]=new_pairs[:,0]
                        adj[:,1]=new_pairs[:,1]
                        #adj[:,2]= random.choices([1,10], weights = [3, 6], k = nc*nv)
                        #adj[:,2]np.random.randint(low = 1,high=6, size=nc*nv)
                        adj[:,2]= random.choices([1,2,3,4,5,6,7,8,9,10], weights = [1,1,2,2,3,3,4,4,5,5], k = nc*nv)
                        #adj[:,2]= random.choices([1,100], k = nc*nv)

                        os.system("mkdir "+path+str(graph)); pickle.dump([adj,nv,k],open(path+str(graph)+'/data','wb'))
                        np.savetxt(path+str(graph)+'/data.txt', adj, fmt='%10d' ,header=str(nv)+'   '+str(nv*nc)+'   '+str(k))

200 20 skewed[1-10] 10


#### LP for feature computation

In [2]:
import xpress as xp
from termcolor import colored
import timeit
import numpy as np
import pickle
import random
def kmedian_xpress_solver_LP(set_path,data,c,n,k):
    print("Inside kmedian",n,k)
    S1 = timeit.default_timer()
    x = np.array ([xp.var (vartype = xp.continuous,ub=1.0) for i in range(n*n)]).reshape (n,n)
    y = np.array([xp.var (vartype = xp.continuous,ub=1.0) for i in range(n)])
    #x = np.array ([xp.var (vartype = xp.free) for i in range(n*n)]).reshape (n,n)
    #y = np.array([xp.var (vartype = xp.free) for i in range(n)])
    print(x.shape,y.shape)
    
    c1=[xp.Sum(y)==k]
    c2=xp.Dot(x,np.ones([n,1]))==np.ones([n,1])
    c3=xp.Dot(np.concatenate([-np.ones([n,1]),np.identity(n)],axis=1),np.concatenate([y.reshape(1,n),x],axis=0))<=np.zeros([n,n])

    #c4=0<=x; c5=x<=1;  c6=0<=y;  c7=y<=1

    profit=xp.Sum(xp.Sum(c*x))

    p=xp.problem()
    p.addVariable(x,y)
    p.setObjective (profit, sense=xp.minimize)
    try:
        p.addConstraint(c1,c2,c3)#,c4,c5,c6,c7)
    except xp.ModelError as e:
      print ("Modeling error:", repr(e))
    p.solve()
    y=p.getSolution(y)
    x=p.getSolution(x)
    E1 = timeit.default_timer()
    #################################################################################################
    median_indices=np.where(y>0.0)[0]
    print(colored('            Time: ','red'), E1 - S1) 
    print(colored('            objective value: ','red'),p.getSolution(profit))
    #print(colored('median indices: ','red'),median_indices+1,median_indices.shape)
     #------------------------------------------------------
    pickle.dump([median_indices,p.getSolution(profit),E1-S1],open(set_path+'/lp_inf','wb'))
    return median_indices,x,y

#### Computing LP based features

In [3]:
def features_related_LP(set_path,median_indices,x,y,c,n):
    #------------------lp_kmedian----------------------
    ind=np.zeros(n);ind[median_indices]=1
    pickle.dump(ind,open(set_path+'/lp_kmedian','wb'))
    pickle.dump(y,open(set_path+'/yf','wb'))
    #--------------------alfa_f-------------------------
    alfa_f=np.zeros(n)
    for i in range(n):
        alfa_f[i]= sum(x[:,i])
    pickle.dump(alfa_f,open(set_path+'/alfa_f','wb'))
    #---------------------µf-------------------------
    µf=np.zeros(n);
    for i in range(n):
        if alfa_f[i]!=0:
            µf[i]= sum(x[:,i]*c[:,i])/alfa_f[i]
    pickle.dump(µf,open(set_path+'/µf','wb'))
    #---------------------Vf,t(t=2,4,8)-------------------------
    T=[2,4,8]
    for t in T:
        V_ft=np.zeros(n);
        for i in range(n):
            V_ft[i]=len(np.where(c[:,i]<=(t*µf[i]))[0])
        pickle.dump(V_ft,open(set_path+'/V_f_'+str(t),'wb'))
    #---------------------Bf,t(t=2,4,8)-------------------------
    dav=np.zeros(n)
    for i in range(n):
        dav[i]=sum(c[:,i]*x[:,i])

    T=[2,4,8]
    for t in T:
        B_ft=np.zeros(n)
        for i in range(n):
            B_ft[i]=len(np.where(c[:,i]<=(t*dav[i]))[0])
        pickle.dump(B_ft,open(set_path+'/B_f_'+str(t),'wb'))
                    
    #----------------------------Yf----------------------------
    Yf=np.zeros(n)
    for i in range(n):
        Yf[i]=(len(np.where(x[:,i]!=0)[0]))
    pickle.dump(Yf,open(set_path+'/Yf','wb'))

#### Computing features not based on LP

In [4]:
def features_no_related_LP(Mypath,cost_mtr,nv):
    #--------------------------------------------------------
    degree=np.zeros(nv)
    for j in range(nv):
        s=np.where(cost_mtr[:,j]!=penalty)[0]
        degree[j]=sum(cost_mtr[s,j])
    pickle.dump(degree,open(Mypath+'/degree','wb'))
    #--------------------------------------------------------
    T=[2,4,8]
    for t in T:
        T_ft=np.zeros(nv);
        for i in range(nv):
            T_ft[i]=len(np.where(cost_mtr[:,i]<=t)[0])

        pickle.dump(T_ft,open(Mypath+'/T_f_'+str(t),'wb'))
    #--------------------------------------------------------
    min_c=np.zeros(nv)
    for i in range(nv):
        min_c[i]=min(cost_mtr[:,i])
    pickle.dump(min_c,open(Mypath+'/min_c','wb'))
    #---------------------------min_f-----------------------------
    min_f=np.zeros(nv)
    for i in range(nv):
        min_f[i]=min(cost_mtr[:,i])
    pickle.dump(min_f,open(Mypath+'/min_f','wb'))
    #----------------------------Yf,p=1.5----------------------------
    Yf=np.zeros(nv)
    for i in range(nv):
        Yf[i]=len(np.where(cost_mtr[:,i]<=(1.5*min_f[i]))[0])
    pickle.dump(Yf,open(Mypath+'/Yfp','wb'))
    #------------------------------------------------------------------------------------
    s=np.sort(random.sample(range(nv), int(nv/2)))
    s1=np.zeros(nv);s1[s]=1
    ss=[]
    for i in range(nv):
        if i not in s:
            ss.append(i)
    ss=np.array(ss);s2=np.zeros(nv);s2[ss]=1
    pickle.dump(s1,open(Mypath+'/f1','wb'))
    pickle.dump(s2,open(Mypath+'/f2','wb'))

#### Calling feature computation (LP-based and non LP-based) after computing LP

In [5]:
for Mode in modes:
    print(Mode,'--------------------------')
    for nv in nV:
        for nc in nC:
            print('------------------------------------------')
            for k in K:
                print('------------------------------------------')
                for d in da:
                    print('--- nV=',nv,' nC=',nc,' k=',k, ' mode=',d)
                    for graph in range(num_graphs):
                        print('       data=',graph)
                        Mypath='data/'+Mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/'+d+'/'+str(graph)
                        data = pickle.load(open(Mypath+'/data','rb'))
                        nv= data[1]; k=data[2]; data=data[0]
                        data=data.astype(int)
                        data[:,0]=data[:,0]-1
                        data[:,1]=data[:,1]-1
                        cost_mtr=np.zeros([nv,nv])+penalty
                        for i in range (len(data)):
                            cost_mtr[data[i,1],data[i,0]]=data[i,2]
                        #print("Calling kmedian with ",Mypath,data.shape,cost_mtr.shape,nv,k)
                        median_indices,x,y=kmedian_xpress_solver_LP(Mypath,data,cost_mtr,nv,k)
                        features_related_LP(Mypath,median_indices,x,y,cost_mtr,nv)
                        features_no_related_LP(Mypath,cost_mtr,nv)

skewed[1-10] --------------------------
------------------------------------------
------------------------------------------
--- nV= 200  nC= 20  k= 10  mode= testing
       data= 0
Inside kmedian 200 10
(200, 200) (200,)
FICO Xpress v8.10.0, Hyper, solve started 0:04:43, Aug 13, 2021
Heap usage: 19MB (peak 19MB, 1334KB system)
Minimizing LP noname
Original problem has:
     40201 rows        40200 cols       120200 elements
Presolved problem has:
     40201 rows        40200 cols       120200 elements
Presolve finished in 0 seconds
Heap usage: 26MB (peak 40MB, 1335KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+01] / [ 1.00e+00,  1.00e+01]
  Objective      [min,max] : [ 1.00e+00,  1.00e+04] / [ 1.00e+00,  1.00e+04]
Autoscaling applied standard scaling

Crash basis containing 200 structural columns created
Starting parall

#### Run ILP to get labelled training data and evaluation on test data

In [ ]:
import xpress as xp
from termcolor import colored
import timeit
import math
import numpy as np
import pickle
import sys
import matplotlib.pyplot as plt
import networkx as nx

def floyd_warshall(G,nV):
    distance = list(map(lambda i: list(map(lambda j: j, i)), G))

    # Adding vertices individually
    for k in range(nV):
        for i in range(nV):
            for j in range(nV):
                distance[i][j] = min(distance[i][j], distance[i][k] + distance[k][j])
    return (distance)

def kmedian_xpress_solver_ILP(data,n,k):

    c=np.zeros([n,n])+penalty
    data=data.astype(int)
    data[:,0]=data[:,0]-1
    data[:,1]=data[:,1]-1
    for i in range (len(data)):
        c[data[i,1],data[i,0]]=data[i,2]
    #c=floyd_warshall(c,n)

    S=range(n);SS=range(n*n)
    x = np.array ([xp.var (vartype = xp.binary) for i in SS]).reshape (n,n)
    y = np.array([xp.var (vartype = xp.binary) for i in S])

    c1=[xp.Sum(y)==k]
    c2=xp.Dot(x,np.ones([n,1]))==np.ones([n,1])
    c3=xp.Dot(np.concatenate([-np.ones([n,1]),np.identity(n)],axis=1),np.concatenate([y.reshape(1,n),x],axis=0))<=np.zeros([n,n])

    profit=xp.Sum(xp.Sum(c*x))

    p=xp.problem()
    p.addVariable(x,y)
    p.setObjective (profit, sense=xp.minimize)
    p.addConstraint(c1,c2)
    p.addConstraint(c3)
    p.solve()

    median_indices=np.where(p.getSolution(y)>=.5)[0]
    return median_indices,p.getSolution(profit)


for nv in nV:
    for nc in nC:
        for k in K:
            for mode in modes:
                for d in da:
                    path='data/'+mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/'+d+'/'
                    for m in range(num_graphs):
                        print(colored('....................................................','blue'),m)
                        S = timeit.default_timer()
                        set_path=path+str(m)
                        data = pickle.load(open(set_path+'/data','rb'))
                        #print(data[0],data[1],data[2])

                        labels=kmedian_xpress_solver_ILP(data[0],data[1],data[2])

                        E = timeit.default_timer() 
                        pickle.dump([labels,E-S],open(set_path+'/points_inf_xp','wb'))

                        print(data[1],data[2],data[0].shape)
                        print(colored('objective value: ','red'),labels[1])
                        print(E - S) 
                        print(colored('median indices: ','red'),labels[0]+1,len(labels[0]))

.................................................... 0
FICO Xpress v8.10.0, Hyper, solve started 0:05:01, Aug 13, 2021
Heap usage: 20MB (peak 20MB, 10069KB system)
Minimizing MILP noname
Original problem has:
     40201 rows        40200 cols       120200 elements     40200 globals
Presolved problem has:
     40201 rows        40200 cols       120200 elements     40200 globals
Presolve finished in 0 seconds
Heap usage: 33MB (peak 47MB, 10071KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+01] / [ 1.00e+00,  1.00e+01]
  Objective      [min,max] : [ 1.00e+00,  1.00e+04] / [ 1.00e+00,  1.00e+04]
Autoscaling applied standard scaling

Will try to keep branch and bound tree memory usage below 88.2GB
 *** Heuristic solution found:  1800151.000      Time: 1 ***
Starting concurrent solve with dual, primal and barrier (30 threads)

 

3675271       51496.40442      D   9865     0        .000000   702
3676801       62072.79948      P      0     0        .000000   704
Optimal solution found
Dual solved problem
  3676801 simplex iterations in 704s

Final objective                       : 6.207279947828199e+04
  Max primal violation      (abs/rel) :       0.0 /       0.0
  Max dual violation        (abs/rel) :       0.0 /       0.0
  Max complementarity viol. (abs/rel) :       0.0 /       0.0
 
Starting root cutting & heuristics
 
 Its Type    BestSoln    BestBound   Sols    Add    Del     Gap     GInf   Time
   1  K   580897.0000  79948.77705      7      5      0   86.24%    2547    753
   2  K   580897.0000  79948.77705      7      3      5   86.24%    2595    757
   3  K   580897.0000  79948.77705      7      3      3   86.24%    2534    760
   4  K   580897.0000  79948.77705      7      2      4   86.24%    2489    764
   5  K   580897.0000  79948.77705      7      2      2   86.24%    2564    767
   6  K   580897.0

    8773  500912.0000  81609.45449      9   5055    310   83.71%    1466   1479
    8873  500912.0000  81609.45449      9   5247    353   83.71%    1281   1485
    8973  500912.0000  81609.45449      9   5431    191   83.71%    1818   1492
Heap usage: 2574MB (peak 2574MB, 17MB system)
B&B tree size: 133MB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
    9073  500912.0000  81609.45449      9   5519     53   83.71%    2296   1499
    9173  500912.0000  81609.45449      9   5580    350   83.71%    1297   1507
    9273  500912.0000  81609.45449      9   5636    324   83.71%    1580   1514
    9373  500912.0000  81609.45449      9   5683    323   83.71%    1779   1523
    9473  500912.0000  84098.68959      9   5755    355   83.21%    1396   1532
    9573  500912.0000  84098.68959      9   5926    362   83.21%    1364   1541
    9673  500912.0000  84098.68959      9   5959    197   83.21%    1937   1551
    9773  500912.0000  84098.68959      9   6

   63107  500912.0000  148847.0911      9  54173     71   70.28%    1510   4725
   64108  500912.0000  148847.0911      9  55200    143   70.28%     918   4757
   65108  500912.0000  148847.0911      9  55671     95   70.28%    1323   4787
   66108  500912.0000  148988.3951      9  56796    124   70.26%    1163   4815
   67108  500912.0000  148988.3951      9  57782     43   70.26%    1792   4844
   68109  500912.0000  148988.3951      9  58607    120   70.26%    1722   4870
   69109  500912.0000  149835.1361      9  59641    141   70.09%    1067   4895
   70109  500912.0000  149835.1361      9  60384     48   70.09%    1582   4920
   71109  500912.0000  149881.7581      9  61615    110   70.08%    1321   4944
   72109  500912.0000  149881.7581      9  62582    154   70.08%     991   4975
Heap usage: 8471MB (peak 8471MB, 26MB system)
B&B tree size: 1.7GB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
   73110  500912.0000  149881.7581      9  63

Heap usage: 10GB (peak 10GB, 37MB system)
B&B tree size: 4.0GB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
  187538  470952.0000  177602.6476     11 151007     95   62.29%    1451  11593
Heuristic R running (150000 LP iterations in 1085.3 seconds)
Heuristic R running (50000 LP iterations in  246.8 seconds)
Heuristic R running (100000 LP iterations in  551.8 seconds)
Heuristic R running (50000 LP iterations in  199.7 seconds)
Heuristic R running (50000 LP iterations in  140.2 seconds)
Heuristic R running (50000 LP iterations in  143.5 seconds)
Heuristic R running (50000 LP iterations in  144.2 seconds)
Heuristic R running (50000 LP iterations in  127.3 seconds)
Heuristic R running (50000 LP iterations in  241.8 seconds)
Heuristic R running (50000 LP iterations in  216.0 seconds)
Heuristic R running (50000 LP iterations in  232.1 seconds)
Heuristic R running (50000 LP iterations in  152.9 seconds)
Heuristic R running (100000 LP iterations in  7

Heuristic R running (50000 LP iterations in  150.8 seconds)
Heuristic R running (100000 LP iterations in  448.9 seconds)
Heuristic R running (50000 LP iterations in  216.3 seconds)
Heuristic R running (50000 LP iterations in  212.0 seconds)
Heuristic R running (50000 LP iterations in  398.1 seconds)
Heuristic R running (50000 LP iterations in  310.8 seconds)
Heuristic R running (50000 LP iterations in  312.4 seconds)
  307544  470952.0000  191316.6308     11 258990     74   59.38%    1501  22231
Heuristic R running (50000 LP iterations in  331.8 seconds)
Heuristic R running (50000 LP iterations in  235.5 seconds)
Heuristic R running (50000 LP iterations in  154.3 seconds)
Heuristic R running (50000 LP iterations in  171.0 seconds)
Heuristic R running (100000 LP iterations in  650.2 seconds)
Heuristic R running (50000 LP iterations in  316.9 seconds)
Heuristic R running (100000 LP iterations in  695.1 seconds)
Heuristic R running (100000 LP iterations in  609.9 seconds)
Heuristic R runn

Heuristic R running (100000 LP iterations in  525.6 seconds)
Heuristic R running (50000 LP iterations in  242.7 seconds)
Heuristic R running (50000 LP iterations in  222.8 seconds)
Heuristic R running (50000 LP iterations in  249.0 seconds)
Heuristic R running (100000 LP iterations in  702.4 seconds)
Heuristic R running (150000 LP iterations in  900.9 seconds)
Heuristic R running (100000 LP iterations in  515.9 seconds)
Heuristic R running (50000 LP iterations in  320.3 seconds)
Heuristic R running (50000 LP iterations in  239.7 seconds)
Heuristic R running (50000 LP iterations in  240.3 seconds)
Heuristic R running (50000 LP iterations in  435.1 seconds)
  440185  461045.0000  201346.8439     12 368078     82   56.33%    1803  32788
Heuristic R running (50000 LP iterations in  347.5 seconds)
Heuristic R running (50000 LP iterations in  238.7 seconds)
Heuristic R running (50000 LP iterations in  140.8 seconds)
Heuristic R running (100000 LP iterations in 1003.7 seconds)
Heuristic R run

Heuristic R running (150000 LP iterations in 1427.4 seconds)
Heuristic R running (50000 LP iterations in  134.6 seconds)
  577022  441013.0000  209274.9995     13 444523    108   52.55%    1020  42305
Heuristic R running (50000 LP iterations in  329.6 seconds)
Heuristic R running (50000 LP iterations in  224.4 seconds)
Heuristic R running (50000 LP iterations in  135.7 seconds)
Heuristic R running (50000 LP iterations in  150.1 seconds)
Heuristic R running (50000 LP iterations in  121.4 seconds)
Heuristic R running (50000 LP iterations in  215.8 seconds)
Heuristic R running (100000 LP iterations in  482.8 seconds)
Heuristic R running (50000 LP iterations in  212.5 seconds)
Heuristic R running (50000 LP iterations in  123.2 seconds)
Heap usage: 18GB (peak 18GB, 32MB system)
B&B tree size: 12.1GB total
               14MB active node information
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
  587023  441013.0000  211191.9068     13 453297     87   52.1

#### Load libraries to run Classification model

In [ ]:
import numpy as np
import sys
import pickle
import re
import matplotlib.pyplot as plt
from timeit import default_timer as timer
from termcolor import colored
import xpress as xp
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report,precision_recall_curve, confusion_matrix, accuracy_score
import xgboost
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import roc_curve
from sklearn import svm

#### Load features for classification

In [ ]:
def feature_loader(set_path): 
    f0=pickle.load(open(set_path+'/alfa_f','rb'))
    f1=pickle.load(open(set_path+'/B_f_2','rb'))
    f2=pickle.load(open(set_path+'/B_f_4','rb'))
    f3=pickle.load(open(set_path+'/B_f_8','rb'))
    f4=pickle.load(open(set_path+'/lp_kmedian','rb'))
    f5=pickle.load(open(set_path+'/min_f','rb'))
    f6=pickle.load(open(set_path+'/V_f_2','rb'))
    f7=pickle.load(open(set_path+'/V_f_4','rb'))
    f8=pickle.load(open(set_path+'/V_f_8','rb'))
    f9=pickle.load(open(set_path+'/yf','rb'))
    f10=pickle.load(open(set_path+'/Yf','rb'))
    f11=pickle.load(open(set_path+'/µf','rb'))
    f12=pickle.load(open(set_path+'/Yfp','rb'))
#     f13=pickle.load(open(set_path+'/degree','rb'))
    f14=pickle.load(open(set_path+'/T_f_2','rb'))
    f15=pickle.load(open(set_path+'/T_f_4','rb'))
    f16=pickle.load(open(set_path+'/T_f_8','rb'))
#     f17=pickle.load(open(set_path+'/min_c','rb'))
#     f18=pickle.load(open(set_path+'/f1','rb'))
#     f19=pickle.load(open(set_path+'/f2','rb'))
#     f20=pickle.load(open(set_path+'/lp_prime','rb'))
#     f21=pickle.load(open(set_path+'/NLP1','rb'))
    #f22=pickle.load(open(set_path+'/NILP','rb'))
    #f23= pickle.load(open(set_path+'/ga_ind','rb'))
    
    features=np.vstack([f0[:],f1[:],f2[:],f3[:],f4[:],f5[:],f6[:],f7[:],f8[:],f9[:],f10[:],f11[:],f12[:],f14[:],f15[:],f16[:]]);features=features.transpose()
    #features=np.vstack([f0[:],f1[:],f2[:],f3[:],f4[:],f5[:],f6[:],f7[:],f8[:],f9[:],f10[:],f11[:]]);features=features.transpose()
    #features=np.vstack([f4[:],f10[:],f7[:],f2[:]]);features=features.transpose()
    
    
    return(features)

#### Load ground truth for classification

In [ ]:
path='data/'+modes[0]+'/V='+str(nV[0])+'-C='+str(nC[0])+'/k='+str(K[0])+'/training/0'
y=pickle.load(open(path+'/points_inf_xp','rb'))
y_true=np.zeros(nV[0]);y_true[y[0][0]]=1
features=feature_loader(path)


for Mode in modes:
    for nv in nV:
        for nc in nC:
            for k in K:
                for i in range(1,num_graphs):
                    Mypath='data/pm_'+str(nV[0])+'/'+Mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/training/'+str(i)
                    y=pickle.load(open(Mypath+'/points_inf_xp','rb'))
                    y_temp=np.zeros(nV[0]);y_temp[y[0][0]]=1
                    y_true=np.hstack([y_true,y_temp])
                    f_temp=feature_loader(Mypath)
                    features=np.vstack([features,f_temp])
print(len(y_true),len(features))


#### Normalisation and Learning Classification Model

In [ ]:
sc = StandardScaler()
features = sc.fit_transform(features)


clf = RandomForestClassifier(max_depth=5,n_estimators=100,min_samples_split=10, random_state=0,class_weight="balanced")
# clf = xgboost.XGBClassifier(n_estimators=100, learning_rate=1.0, max_depth=3, random_state=0, class_weight='balanced')
# clf=AdaBoostClassifier(n_estimators=100, random_state=0)
# clf=svm.SVC(cache_size=200, class_weight='balanced',random_state=None)

clf.fit(features,y_true)

print(colored('Feature importance:','blue'))
print(clf.feature_importances_)

#### Output the learnt model to a file

In [ ]:
for Mode in modes:
    for nv in nV:
        for nc in nC:
            for k in K:
                for i in range(num_graphs):
                    test_path = 'data/'+Mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/testing/'+str(i)

                    test_features=feature_loader(test_path)
                    test_features = sc.transform(test_features)

                    y_pred = clf.predict(test_features)

                    probs= clf.predict_proba(test_features)

                    pickle.dump(np.where(y_pred==1)[0],open(test_path+'/pruning_points_'+str(i),'wb'))
                    pickle.dump(probs[:,1],open(test_path+'/probs','wb'))
                    pickle.dump(np.where(y_pred==1)[0],open(test_path+'/probs_xgb','wb'))


                    print(len(np.where(y_pred==1)[0]))

#### Compute the ILP on pruned data set (pruning based on different thresholds)

In [ ]:
import numpy as np
import sys
import pickle
import re
import matplotlib.pyplot as plt
from timeit import default_timer as timer
from termcolor import colored
import xpress as xp
from sklearn.metrics import confusion_matrix
import random


def kmedian_xpress_solver_pruned(points,n,c,num_fac,k):
    S=range(n);SS=range(n*num_fac)
    x = np.array ([xp.var (vartype = xp.binary) for i in SS]).reshape (n,num_fac)
    y = np.array([xp.var (vartype = xp.binary) for i in range(num_fac)])

    c1=[xp.Sum(y)==k]
    c2=xp.Dot(x,np.ones([num_fac,1]))==np.ones([n,1])
    c3=xp.Dot(np.concatenate([-np.ones([n,1]),np.identity(n)],axis=1),np.concatenate([y.reshape(1,num_fac),x],axis=0))<=np.zeros([n,num_fac])

    profit=xp.Sum(xp.Sum((c*x)))
    
    p=xp.problem()
    p.addVariable(x,y)
    p.setObjective (profit, sense=xp.minimize)
    p.addConstraint(c1,c2,c3)
    p.solve()
    
    #print(colored('--------------','green'))
    #print(p.getSolution(y))
    #print(colored('--------------','green'))
    
    median_indices=np.where(p.getSolution(y)>=0.5)[0]
    #print(p.getSolution(y),k)
    #print(median_indices)

    return median_indices,p.getSolution(profit)


#///////////////////////////////////////////////////////////////////////////////////////////////////
for nv in nV:
    for nc in nC:
        for Mode in modes:
            for k in K:
                path='data/'+Mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/testing/'
                thre = [k,min(int(1.5*k),nv),min(int(2*k),nv),min(int(3*k),nv),min(int(4*k),nv),int(nv/2),int(nv)]
                value_n=np.zeros(len(thre))
                Time=np.zeros(len(thre))
                for m in range(num_graphs):
                    print(colored('....................................................','blue'),m)
                    set_path=path+str(m)
                    data = pickle.load(open(set_path+'/data','rb'))

                    probs=pickle.load(open(set_path+'/probs','rb'))
                    ind=np.argsort(probs)


                    n=data[1];k=data[2]
                    data=data[0].astype(int)
                    data[:,0]=data[:,0]-1
                    data[:,1]=data[:,1]-1

                    for th_index,th in enumerate(thre):
                        ind_last_thre = ind[len(ind)-th:]
                        num_fac = len(ind_last_thre)

                        c=np.zeros([n,len(ind_last_thre)])
                        for i in range(n):
                            for j in range(len(ind_last_thre)):
                                    c[i,j]=penalty

                        for i in range (len(ind_last_thre)):
                            mp=(np.where(data[:,0]==ind_last_thre[i])[0])
                            c[data[mp,1],i]=data[mp,2]

                        #print(colored("--------------------",'blue'))
                        #print(data,n,c,num_fac,k)
                        #print(colored("--------------------",'blue'))
                        S=timer()
                        median_indices,value=kmedian_xpress_solver_pruned(data,n,c,num_fac,k)
                        E=timer()
                        print(colored('pruned centers:','red'),ind_last_thre+1)
                        print(colored('selected centers:','red'),np.sort(ind_last_thre[(median_indices)])+1,len(median_indices))
                        print(colored('Objective Value:','red'),value)
                        print(E-S)

                        s=pickle.load(open(set_path+'/points_inf_xp','rb'))
                        s=s[0][1]
                        value_n[th_index]=value/s
                        Time[th_index]=E-S
                        #print(s,value_n[thi],value)
                    print(colored("Value_n: " +str(value_n),'red'))
                    print(colored("Time: "+str(Time),'red'))

#### Add a constraint that at least 80% of nodes to be taken from pruned set

In [ ]:
import numpy as np
import sys
import pickle
import re
import matplotlib.pyplot as plt
from timeit import default_timer as timer
from termcolor import colored
import xpress as xp
from sklearn.metrics import confusion_matrix
import random


def kmedian_xpress_solver_constrained(points,n,c,fac_index,k):
    S=range(n);SS=range(n*n)
    x = np.array ([xp.var (vartype = xp.binary) for i in SS]).reshape (n,n)
    y = np.array([xp.var (vartype = xp.binary) for i in range(n)])

    c1=[xp.Sum(y)==k]
    c2=xp.Dot(x,np.ones([n,1]))==np.ones([n,1])
    c3=xp.Dot(np.concatenate([-np.ones([n,1]),np.identity(n)],axis=1),np.concatenate([y.reshape(1,n),x],axis=0))<=np.zeros([n,n])
    c4=[xp.Sum(y[fac_index])>=int(0.8*k)]

    profit=xp.Sum(xp.Sum((c*x)))
    
    p=xp.problem()
    p.addVariable(x,y)
    p.setObjective (profit, sense=xp.minimize)
    p.addConstraint(c1,c2,c3,c4)
    p.solve()
    
    #print(colored('--------------','green'))
    #print(p.getSolution(y))
    #print(colored('--------------','green'))
    
    median_indices=np.where(p.getSolution(y)>=0.5)[0]
    #print(p.getSolution(y),k)
    #print(median_indices)

    return median_indices,p.getSolution(profit)


#///////////////////////////////////////////////////////////////////////////////////////////////////
for nv in nV:
    for nc in nC:
        for Mode in modes:
            for k in K:
                path='data/'+Mode+'/V='+str(nv)+'-C='+str(nc)+'/k='+str(k)+'/testing/'
                thre = [k,min(int(1.5*k),nv),min(int(2*k),nv),min(int(3*k),nv),min(int(4*k),nv),int(nv/2),int(nv)]
                value_n=np.zeros(len(thre))
                Time=np.zeros(len(thre))
                for m in range(num_graphs):
                    print(colored('....................................................','blue'),m)
                    set_path=path+str(m)
                    data = pickle.load(open(set_path+'/data','rb'))

                    probs=pickle.load(open(set_path+'/probs','rb'))
                    ind=np.argsort(probs)


                    n=data[1];k=data[2]
                    data=data[0].astype(int)
                    data[:,0]=data[:,0]-1
                    data[:,1]=data[:,1]-1

                    for th_index,th in enumerate(thre):
                        ind_last_thre = ind[len(ind)-th:]

                        cost_mtr=np.zeros([n,n])+penalty
                        for i in range (len(data)):
                            cost_mtr[data[i,1],data[i,0]]=data[i,2]

                        #print(colored("--------------------",'blue'))
                        #print(data,n,c,num_fac,k)
                        #print(colored("--------------------",'blue'))
                        S=timer()
                        median_indices,value=kmedian_xpress_solver_constrained(data,n,cost_mtr,ind_last_thre,k)
                        E=timer()
                        print(colored('pruned centers:','red'),ind_last_thre+1)
                        print(colored('selected centers:','red'),np.sort(median_indices)+1,len(median_indices))
                        print(colored('Objective Value:','red'),value)
                        print(E-S)

                        s=pickle.load(open(set_path+'/points_inf_xp','rb'))
                        s=s[0][1]
                        value_n[th_index]=value/s
                        Time[th_index]=E-S
                        #print(s,value_n[thi],value)
                    print(colored("Value_n: " +str(value_n),'red'))
                    print(colored("Time: "+str(Time),'red'))